In [4]:
# Import the os module, for the os.walk function
import pymongo
from pymongo import UpdateOne, WriteConcern
import re
import pandas as pd
import numpy as np
#import wasstraat.meta as meta
#import wasstraat.mongoUtils as mongoUtil
import simplejson
 
# Import app code
# Absolute imports for Hydrogen (Jupyter Kernel) compatibility
import shared.config as config
import logging
logger = logging.getLogger()


fullfilename = '/input/images/DC014_Doelenstraat/L Fotos/Hout/DC14_P00_H0000_701_2.JPG'
dir = '/output/tijdelijk/'

In [7]:
def getAnalyseCollection():   
    myclient = pymongo.MongoClient(str(config.MONGO_URI))
    analyseDb = myclient[str(config.DB_ANALYSE)]
    return analyseDb[config.COLL_ANALYSE]

def getAnalyseCleanCollection():   
    myclient = pymongo.MongoClient(str(config.MONGO_URI))
    analyseDb = myclient[str(config.DB_ANALYSE)]
    return analyseDb[config.COLL_ANALYSE_CLEAN]


def setReferences(soort, col='analyse', key='key', refkey=None):
    try:
        if (col == 'analyse'):
            collection = getAnalyseCollection()
        elif (col == 'analyseclean'):
            collection = getAnalyseCleanCollection()
        else:
            raise ValueError('Error: Herkent de collectie niet met naam ' + col)

        soort_lw = soort.lower()
        
        # Find all main entries for type soort
        soort_query = {'soort': soort}
        if soort == 'Tekening':
            soort_query = {'soort': 'Foto', 'tekeningcd': {'$exists': True}}

        df_soort = pd.DataFrame(list(collection.find(soort_query, projection={key:1, 'ID':1})))
        df_soort = df_soort.rename(columns={'_id': soort_lw+'UUID', 'index':soort_lw+'ID', key:'key_'+soort_lw})
        if df_soort.size < 1:
            logger.warning("Er zjn geen documents gevonden van het type " +soort)
            return

        if not 'key_'+soort_lw in df_soort.columns:
            logger.warning("Kan geen referenties maken voor " +soort + ". Geen Key-veld aanwezig.")
            return

        # Find all references to type soort
        if refkey:
            df_ref = pd.DataFrame(list(collection.find({refkey: {"$exists": True}}, projection={refkey:1})))
            df_ref = df_ref.rename(columns={refkey: 'key_'+soort_lw})
        else:
            df_ref = pd.DataFrame(list(collection.find({"key_"+soort_lw: {"$exists": True}}, projection={'key_'+soort_lw:1})))
        if df_ref.size < 1:
            logger.warning("Er zjn geen referentie met key_"+soort_lw+" gevonden naar documents van het type " +soort )
            return
            
        # Merge dataframes to connect ID's en UUID's to referencing docs
        df_merge = pd.merge(df_ref, df_soort, how='left', on='key_'+soort_lw).rename(columns={'ID': soort_lw + 'ID'})
        
        # Update soort documents 
        updates=[ UpdateOne({'_id':x['_id']}, {'$set':x}) for x in [v.dropna().to_dict() for k,v in df_merge.iterrows()]] # 
        result = collection.bulk_write(updates)

        return result.bulk_api_result
        
    except Exception as err:
        msg = "Onbekende fout bij het aanroepen van een aggregation met melding: " + str(err)
        logger.error(msg)   
        raise Exception(msg) from err
 
    finally:
        collection.database.client.close()


In [9]:
setReferences('ABR', col='analyseclean', refkey='key_abr_materiaal')

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 110094,
 'nModified': 110094,
 'nRemoved': 0,
 'upserted': []}